# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [5]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [44]:
root = document_tree.getroot()
root.tag

root.attrib

for c in root.iter('country'):
     print (c.attrib)

{'area': '28750', 'memberships': 'org-BSEC org-CEI org-CD org-SELEC org-CE org-EAPC org-EBRD org-EITI org-FAO org-IPU org-IAEA org-IBRD org-ICC org-ICAO org-ICCt org-Interpol org-IDA org-IFRCS org-IFC org-IFAD org-ILO org-IMO org-IMF org-IOC org-IOM org-ISO org-OIF org-ITU org-ITUC org-IDB org-MIGA org-NATO org-OSCE org-OPCW org-OAS org-OIC org-PCA org-UN org-UNCTAD org-UNESCO org-UNIDO org-UPU org-WCO org-WFTU org-WHO org-WIPO org-WMO org-UNWTO org-WTO', 'capital': 'cty-Albania-Tirane', 'car_code': 'AL'}
{'area': '131940', 'memberships': 'org-AG org-BIS org-BSEC org-CD org-SELEC org-CE org-EMU org-EAPC org-EBRD org-ECB org-EIB org-CERN org-ESA org-EU org-FATF org-FAO org-IGAD org-IPU org-IAEA org-IBRD org-ICC org-ICAO org-ICJ org-ICCt org-Interpol org-IDA org-IEA org-IFRCS org-IFC org-IFAD org-IHO org-ILO org-IMO org-IMSO org-IMF org-IOC org-IOM org-OIF org-ITSO org-ITU org-ITUC org-MIGA org-NATO org-NEA org-NSG org-OECD org-OSCE org-OPCW org-OAS org-PCA org-UN org-UNCTAD org-UNESCO o

In [171]:
countries = document_tree.findall('country')
print( document_tree.findall('country').[len(countries)-1].find('name').text)

SyntaxError: invalid syntax (<ipython-input-171-e203a0226353>, line 2)

In [46]:
for child in root.findall('.'):
    print(child.tag)

mondial


In [32]:
root[0][0].text + " " + root[0][1].text + " " + root[0][2].text 

'Albania 1214489 1618829'

In [53]:
list(root[0].iter("."))

[]

In [22]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [71]:
document_tree.find('country').find('infant_mortality').text

'13.19'

In [65]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
document = ET.parse( './data/mondial_database.xml' )

In [3]:
import pandas as pd
import numpy as np

### 1. Find 10 countries with the lowest infant mortality rates

In [128]:
# store the results in a data frame with one column for country name and another for infant mortality rates
n_country = len(document.find('country')) 

store_mortality = pd.DataFrame({"name": [np.nan] * (n_country +1),
                                "infant_mortality": [np.nan] * (n_country +1)})
#store_mortality.head()

,infant_mortality,name
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [133]:
# iterate over all the countries to find infant mortality rates
i = 0
for element in document.iterfind('country'):
    store_mortality.name[i] = element.find('name').text
    
    inf_mor = element.find('infant_mortality') # It should be "xml.etree.ElementTree.Element" unless missing value
    
    if (str(inf_mor) != 'None'):
        store_mortality.infant_mortality[i] = float(inf_mor.text)
    i += 1
    

In [134]:
# list the 10 lowest infant mortality countries
store_mortality.dropna().sort_values("infant_mortality")[:10]

,infant_mortality,name
38,1.81,Monaco
36,2.48,Norway
37,2.60,Sweden
10,2.63,Czech Republic
7,3.31,France
13,3.31,Italy
8,3.33,Spain
35,3.36,Finland
11,3.46,Germany
18,3.64,Belarus


### 2. Find 10 cities with the largest population

In [206]:
# first initiate some empty lists to store information
cities = []
years = []
pops = []

# iterate through countries
for e in document.iterfind('country'):
    
    # interate through cities
    for c in e.getiterator('city'):
        
        # iterate through all population data years
        for c_pop in c.getiterator('population'):  
            
            #store city name, year, and population
            cities += [c.find('name').text]
            years += [c_pop.attrib.get('year')]
            pops += [c_pop.text]
            
# create a data frame
pop_city = pd.DataFrame({"city_name": pd.Categorical(cities), 
                          "Year": pd.Series(years, dtype = 'int32'),
                          "Population": pd.Series(pops, dtype= 'int32')})

pop_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9320 entries, 0 to 9319
Data columns (total 3 columns):
Population    9320 non-null int32
Year          9320 non-null int32
city_name     9320 non-null category
dtypes: category(1), int32(2)
memory usage: 114.5 KB


In [207]:
# organize the data frame as a pivot table with years and cities
spread_pop = pd.pivot_table(pop_city, values='Population', index=['Year'], columns=['city_name'])
spread_pop

city_name,'s-Hertogenbosch,A Coruña,Aachen,Aalborg,Aba,Abadan,Abakan,Abeokuta,Aberdeen,Abidjan,...,České Budějovice,Ėngel's,İnegöl,Łódź,Şanlıurfa,Šachty,Ščëlkovo,Železnodorožnyj,Žilina,Žukovskij
Year,,,,,,,,,,,,,,,,,,,,,
1950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It looks like a mess in that each city has a different set of years with available population data.
As a result, we cannot find a common year such that all cities' populations could be compared.

In [213]:
# Doing the best we can given this dataset, interpolate the population according to years
pop_fillna = spread_pop.copy()

pop_fillna = pop_fillna.interpolate(method = 'index')

pop_fillna

city_name,'s-Hertogenbosch,A Coruña,Aachen,Aalborg,Aba,Abadan,Abakan,Abeokuta,Aberdeen,Abidjan,...,České Budějovice,Ėngel's,İnegöl,Łódź,Şanlıurfa,Šachty,Ščëlkovo,Železnodorožnyj,Žilina,Žukovskij
Year,,,,,,,,,,,,,,,,,,,,,
1950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It is not ideal that countries were assume a constant population after the year of its last available data.
I will choose a year with the most number of available data across all cities to compare their population.

In [215]:
# find the years with the most available data

pop_city.Year.value_counts().head() # 2010 it is

2010    1015
1990     972
2000     964
1991     784
2011     647
Name: Year, dtype: int64

In [245]:
# list the highest population in 2010 in descending orders
pop_fillna.iloc[41,:].sort_values(ascending = False)[:10]

city_name
Shanghai     2.231547e+07
Istanbul     1.289267e+07
Mumbai       1.238958e+07
Beijing      1.171662e+07
Moskva       1.161288e+07
Delhi        1.121885e+07
São Paulo    1.115234e+07
Tianjin      1.109031e+07
Guangzhou    1.107142e+07
Shenzhen     1.035838e+07
Name: 2010, dtype: float64

### 3. Find 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

Use the latest population data of each country as the base, and multiply it by the percentage of each ethnic group to be finally summed across all countries. 
First create a data frame of country name, latest population, ethnic group name, and population percentage,
then compute a column wide multiplication.
The last step is to aggregate population by ethnic groups.

In [6]:
# create an empty list to store the results
output = []

# iterate through countries
for e in document.iterfind('country'):  
              
    # iterate through all ethnic groups
    for ethnic in e.getiterator('ethnicgroup'):  

        #get country name
        country_name = str(e.find('name').text)
        
        # get all population elements
        pops = e.findall('population')
        latest_pop = str(pops[-1].text) # take the text of the last population element
        
        # get the percentage of population for this thnic group
        eth_perc = float(ethnic.attrib.get('percentage'))
        
        # find the ethnicity name 
        ethnicity = str(ethnic.text)
        
        # put it all together in the list
        output += [[country_name, latest_pop, ethnicity, eth_perc]]

# create a data frame
ethnicity_df = pd.DataFrame(np.array(output), 
                            columns = ["country_name", "latest_pop", "ethnicity", "eth_perc"])

ethnicity_df.head()

,country_name,latest_pop,ethnicity,eth_perc
0,Albania,2800138,Albanian,95.0
1,Albania,2800138,Greek,3.0
2,Greece,10816286,Greek,93.0
3,Macedonia,2059794,Macedonian,64.2
4,Macedonia,2059794,Albanian,25.2


In [11]:
# compute the population of each ethnic group

# first convert data types
ethnicity_df.latest_pop = ethnicity_df.latest_pop.astype(int)
ethnicity_df.eth_perc = ethnicity_df.eth_perc.astype(float)

ethnicity_df = ethnicity_df.assign(group_pop = lambda df: df.latest_pop * df.eth_perc / 100)

ethnicity_df.head()

,country_name,latest_pop,ethnicity,eth_perc,group_pop
0,Albania,2800138,Albanian,95.0,2.660131e+06
1,Albania,2800138,Greek,3.0,8.400414e+04
2,Greece,10816286,Greek,93.0,1.005915e+07
3,Macedonia,2059794,Macedonian,64.2,1.322388e+06
4,Macedonia,2059794,Albanian,25.2,5.190681e+05


In [20]:
# aggregate the group population across all countries
# and then list the top 10 highest population ethnic groups
ethnicity_df.groupby('ethnicity')[['group_pop']].sum().sort_values('group_pop', ascending=False)[:10]

,group_pop
ethnicity,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


### 4. Find the country of a) longest river, b) largest lake and c) airport at highest elevation

First search for the river, lake and airport keywords in the document,find their attribute information, and then identify country name as well as river length, lake area and airport elevation.

In [37]:
for c in document.iter('river'):
     print (c.attrib) #.find('length').text

{'id': 'river-Thjorsa', 'country': 'IS'}
{'id': 'river-Joekulsa_a_Fjoellum', 'country': 'IS'}
{'id': 'river-Glomma', 'country': 'N'}
{'id': 'river-Lagen', 'country': 'N'}
{'id': 'river-Goetaaelv', 'country': 'S'}
{'id': 'river-Klaraelv', 'country': 'N S'}
{'id': 'river-Umeaelv', 'country': 'S'}
{'id': 'river-Dalaelv', 'country': 'S'}
{'id': 'river-Vaesterdalaelv', 'country': 'S'}
{'id': 'river-Oesterdalaelv', 'country': 'S'}
{'id': 'river-Paatsjoki', 'country': 'SF N R'}
{'id': 'river-Ounasjoki', 'country': 'SF'}
{'id': 'river-Kemijoki', 'country': 'SF'}
{'id': 'river-Oulujoki', 'country': 'SF'}
{'id': 'river-Kymijoki', 'country': 'SF'}
{'id': 'river-Kokemaeenjoki', 'country': 'SF'}
{'id': 'river-Vuoksi', 'country': 'SF R'}
{'id': 'river-Themse', 'country': 'GB'}
{'id': 'river-Maas', 'country': 'NL B F'}
{'id': 'river-Loire', 'country': 'F'}
{'id': 'river-Garonne', 'country': 'F E'}
{'id': 'river-Rhone', 'country': 'F CH'}
{'id': 'river-Saone', 'country': 'F'}
{'id': 'river-Doubs', 'co

In [30]:
for c in document.iter('lake'):
     print (c.attrib)

{'id': 'lake-Inarisee', 'country': 'SF'}
{'id': 'lake-Oulujaervi', 'country': 'SF'}
{'id': 'lake-Kallavesi', 'country': 'SF'}
{'id': 'lake-Saimaa', 'country': 'SF'}
{'id': 'lake-Paeijaenne', 'country': 'SF'}
{'id': 'lake-MjoesaSee', 'country': 'N'}
{'id': 'lake-Storuman', 'country': 'S'}
{'type': 'impact', 'id': 'lake-Siljan', 'country': 'S'}
{'id': 'lake-Maelarsee', 'country': 'S'}
{'id': 'lake-Vaenersee', 'country': 'S'}
{'id': 'lake-Vaettersee', 'country': 'S'}
{'id': 'lake-Arresoe', 'country': 'DK'}
{'id': 'lake-LochNess', 'country': 'GB'}
{'id': 'lake-LochLomond', 'country': 'GB'}
{'id': 'lake-Bodensee', 'country': 'A D CH'}
{'id': 'lake-Chiemsee', 'country': 'D'}
{'id': 'lake-StarnbergerSee', 'country': 'D'}
{'id': 'lake-Ammersee', 'country': 'D'}
{'type': 'caldera', 'id': 'lake-LaacherMaar', 'country': 'D'}
{'id': 'lake-Genfer_See', 'country': 'F CH'}
{'id': 'lake-Zurichsee', 'country': 'CH'}
{'id': 'lake-Thunersee', 'country': 'CH'}
{'id': 'lake-Brienzersee', 'country': 'CH'}
{

In [29]:
airports = document.findall('airport')

for c in airports:
     print (c.attrib)

{'city': 'cty-Afghanistan-2', 'iatacode': 'HEA', 'country': 'AFG'}
{'city': 'cty-Afghanistan-Kabul', 'iatacode': 'KBL', 'country': 'AFG'}
{'city': 'cty-Albania-Tirane', 'iatacode': 'TIA', 'country': 'AL'}
{'city': 'cty-Algeria-14', 'iatacode': 'TEE', 'country': 'DZ'}
{'city': 'cty-Algeria-6', 'iatacode': 'BLJ', 'country': 'DZ'}
{'city': 'cty-Algeria-11', 'iatacode': 'BJA', 'country': 'DZ'}
{'city': 'cty-Algeria-19', 'iatacode': 'TMR', 'country': 'DZ'}
{'city': 'cty-Algeria-17', 'iatacode': 'BSK', 'country': 'DZ'}
{'city': 'cty-Algeria-4', 'iatacode': 'CZL', 'country': 'DZ'}
{'city': 'cty-Algeria-7', 'iatacode': 'QSF', 'country': 'DZ'}
{'city': 'cty-Algeria-3', 'iatacode': 'ORN', 'country': 'DZ'}
{'city': 'cty-Algeria-21', 'iatacode': 'GHA', 'country': 'DZ'}
{'city': 'cty-Algeria-5', 'iatacode': 'AAE', 'country': 'DZ'}
{'city': 'cty-Algeria-Algiers', 'iatacode': 'ALG', 'country': 'DZ'}
{'city': 'cty-Algeria-13', 'iatacode': 'TLM', 'country': 'DZ'}
{'city': 'city-Pago-Pago-AMSA-AMSA', 'i

In [31]:
# River, lake and airport all have a country code in their attribute, and so do countries

document.find('country').attrib # the country code attribute is called "car_code" in the dictionary as printed below

{'area': '28750',
 'capital': 'cty-Albania-Tirane',
 'car_code': 'AL',
 'memberships': 'org-BSEC org-CEI org-CD org-SELEC org-CE org-EAPC org-EBRD org-EITI org-FAO org-IPU org-IAEA org-IBRD org-ICC org-ICAO org-ICCt org-Interpol org-IDA org-IFRCS org-IFC org-IFAD org-ILO org-IMO org-IMF org-IOC org-IOM org-ISO org-OIF org-ITU org-ITUC org-IDB org-MIGA org-NATO org-OSCE org-OPCW org-OAS org-OIC org-PCA org-UN org-UNCTAD org-UNESCO org-UNIDO org-UPU org-WCO org-WFTU org-WHO org-WIPO org-WMO org-UNWTO org-WTO'}

In [92]:
# make a data frame of the country code information
country_coding = [] # first initiate a list to store information

for e in document.iterfind('country'):
    name = e.find('name').text
    
    code = e.attrib.get('car_code')
    
    country_coding += [[name, code]]
    
country_df = pd.DataFrame(country_coding, columns= ['country_name','country_code'])
country_df.head()

,country_name,country_code
0,Albania,AL
1,Greece,GR
2,Macedonia,MK
3,Serbia,SRB
4,Montenegro,MNE


In [85]:
# iterate through rivers
river_length = []

for c in document.iterfind('river'):
    
    if (str(type(c.find('length'))) == "<class 'NoneType'>"):
        length = np.nan
    else:
        length = float(c.find('length').text) # get the length
    code = c.attrib.get('country') # get the country code
    name = str(c.attrib.get('id')) # name of the river
    
    river_length += [[code, name , length]]
    
river_length[:5]

[['IS', 'river-Thjorsa', '230'],
 ['IS', 'river-Joekulsa_a_Fjoellum', '206'],
 ['N', 'river-Glomma', '604'],
 ['N', 'river-Lagen', '322'],
 ['S', 'river-Goetaaelv', '93']]

In [83]:
print(len(river_length), len(document.findall('river'))) # verify the length of the data

238 238


In [90]:
# create a data frame
river_df = pd.DataFrame(np.array(river_length), 
                            columns = ["country_code", "river_name", "length"])

river_df.length = river_df.length.astype(float)

river_df.sort_values('length',ascending = False)[:5]

,country_code,river_name,length
174,CO BR PE,river-Amazonas,6448.0
137,CN,river-Jangtse,6380.0
136,CN,river-Hwangho,4845.0
123,R,river-Lena,4400.0
205,RCB ZRE,river-Zaire,4374.0


In [101]:
# now find those countries that have Amazon flowing through
country_df.country_code = country_df.country_code.astype(str)

print(country_df.country_name[country_df.country_code == 'CO'],
     country_df.country_name[country_df.country_code == 'BR'],
     country_df.country_name[country_df.country_code == 'PE'])

141    Colombia
Name: country_name, dtype: object 176    Brazil
Name: country_name, dtype: object 180    Peru
Name: country_name, dtype: object


In [103]:
# iterate through lakes
for l in document.iterfind('lake'):  
    
    if (str(type(l.find('area'))) == "<class 'NoneType'>"):
        area = np.nan
    else:
        area = float(l.find('area').text) # get the area
    code = str(l.attrib.get('country')) # get the country code
    name = str(l.attrib.get('id')) # name of the lake
    
    lake_area += [[code, name, area]]
    


In [120]:
# create a data frame
lake_df = pd.DataFrame(np.array(lake_area), 
                            columns = ["country_code", "lake_name", "area"])

lake_df.area = lake_df.area.astype(float)

biggest = lake_df.sort_values('area',ascending = False)[0:1]
biggest

,country_code,lake_name,area
265,R AZ KAZ IR TM,lake-KaspischesMeer,386400.0


In [122]:
# find the countries that area represented in this coding
country_df.country_name[ country_df.country_code
                        .isin( 
                                biggest.iloc[0,0] # locate the first element of this country_code
                                .split(' ') # seperate out the single string into a list of codes
                             )
                       ]

23          Russia
56            Iran
59    Turkmenistan
63      Azerbaijan
75      Kazakhstan
Name: country_name, dtype: object

In [114]:
airport_ele = []

# iterate through airports
for a in document.iterfind('airport'):  
    
    if (str(type(a.find('elevation').text)) == "<class 'NoneType'>"):
        elevation = np.nan
    else:
        elevation = float(a.find('elevation').text) 
    code = str(a.attrib.get('country')) # get the country code
    city = str(a.attrib.get('city')) # name of the city
    
    airport_ele += [[code, city, elevation]]

In [115]:
print(len(airport_ele), len(document.findall('airport')))


1315 1315


In [123]:
# create a data frame
airport_df = pd.DataFrame(np.array(airport_ele), 
                            columns = ["country_code", "city", "elevation"])

airport_df.elevation = airport_df.elevation.astype(float)

airport_df.sort_values('elevation',ascending = False)[:5]

,country_code,city,elevation
80,BOL,cty-BOL-1,4063.0
219,CN,cty-China-324,4005.0
241,CN,cty-China-281,3963.0
813,PE,cty-Peru-15,3827.0
815,PE,cty-Peru-10,3311.0


In [124]:
# find the country with an airport of the highest elevation
country_df.country_name[ country_df.country_code == 'BOL' ] #Bolivia

175    Bolivia
Name: country_name, dtype: object